In [ ]:
import os 
import xml.etree.ElementTree as ET
from xml.etree import ElementTree
import re
import pandas as pd
import string
from xml.sax.saxutils import escape

In [ ]:
os.chdir('/content/drive/MyDrive/kFLOW/EKAW 2022/TML generation tryouts /Final annotation ')

In [ ]:
m = df[df.columns[5]] == 1

In [ ]:
for csvFile in os.listdir():
 file = pd.read_csv(csvFile)
 filenames= file['Doc name'].tolist()
 if myfile in filenames:
   print("yes")
   print(csvFile)


yes
prevention.csv
yes
intention.csv
yes
cause.csv


In [ ]:
df.columns[6]

'annotation '

In [ ]:
df.loc[m, [df.columns[0], df.columns[1],df.columns[2],df.columns[3]]] = (df.loc[m, [df.columns[2],df.columns[3],df.columns[0], df.columns[1]]].values)

In [ ]:
df.loc[m, 'annotation'] = 1

In [ ]:
df

In [ ]:
for csvFile in os.listdir():
 df = pd.read_csv(csvFile)
 print(csvFile)
 m = df[df.columns[5]] == 2

 df.loc[m, [df.columns[0], df.columns[1],df.columns[2],df.columns[3]]] = (df.loc[m, [df.columns[2],df.columns[3],df.columns[0], df.columns[1]]].values)
 df.loc[m,df.columns[5]] = 1
 df.to_csv('/content/drive/MyDrive/kFLOW/EKAW 2022/TML generation tryouts /well aligned rows/'+csvFile)

cause.csv
notcause.csv
prevention.csv
enable.csv
intention.csv


In [ ]:
def generation_tml():  
  
  for document in os.listdir():
      mytree = ET.parse(document)
      myroot = mytree.getroot()
      final_relations_list=[]
      path_to_csv='/content/drive/MyDrive/kFLOW/EKAW 2022/TML generation tryouts /well aligned rows/'
      
      for csvFile in os.listdir( '/content/drive/MyDrive/kFLOW/EKAW 2022/TML generation tryouts /well aligned rows'):
        
        tmp_relations_list=[]
        
        
        file = pd.read_csv(path_to_csv+csvFile)
        for index, row in file.iterrows():
            
            if row['Doc name'] == document and row[file.columns[6]]==1:
              print(row['Doc name'], row[file.columns[5]], row[file.columns[1]], row[file.columns[2]], row[file.columns[3]], row[file.columns[4]])
            

              tmp_relations_list.append([row[file.columns[1]],row[file.columns[2]],row[file.columns[3]],row[file.columns[4]]])
        if tmp_relations_list:
                final_relations_list.append([csvFile,tmp_relations_list])
      #print(final_relations_list)
      eiid_enhanced_relation_list=[]
      for relation in final_relations_list:
        relation_type=relation[0][:-4]
        for event_pair in relation[1]:
          #print(event_pair,event_pair[1],event_pair[3])
          eid1=event_pair[1]
          eid2=event_pair[3]
          makeinstances_elements=myroot.findall('MAKEINSTANCE')
          
          #print(makeinstances_elements)
          for element in makeinstances_elements:
            #print(element.get('eventID'))
            if element.get('eventID')==eid1:
              eiid1=element.get('eiid')
              #print(eiid1)
            if  element.get('eventID')==eid2:
              eiid2=element.get('eiid')
              #print(eiid2)
          eiid_enhanced_relation_list.append([None,relation_type,eiid1,eiid2])
      SLINKS=[elm.get('lid') for elm in myroot.findall('SLINK')]
      TLINKS=[elm.get('lid') for elm in myroot.findall('TLINK')]
      ALINKS=[elm.get('lid') for elm in myroot.findall('ALINK')]
      link_ids=set(SLINKS+ALINKS+TLINKS)
      link_ids=[int(link_id[1:]) for link_id in link_ids]
      for rel in eiid_enhanced_relation_list:
        rel[0]='l'+str(max(link_ids)+1)
        link_ids.append(max(link_ids)+1)  
      inner_template = string.Template('<RLINK lid="${id}"   relType="${typeR}" eventInstanceID="${eiid1}" relatedEventInstance="${eiid2}"/>')
      inner_contents = [inner_template.substitute(id=id, typeR=typeR, eiid1=eiid1,eiid2=eiid2) for (id, typeR, eiid1,eiid2) in eiid_enhanced_relation_list]
      #print(eiid_enhanced_relation_list)
      for element in inner_contents:
        root = ET.fromstring(element)
        myroot.append(root)
      tree = ET.ElementTree(myroot)
      tree.write('/content/drive/MyDrive/kFLOW/EKAW 2022/TML generation tryouts /TB data/'+document) 
      #print(myroot.findall('RLINK'))
    

In [ ]:
generation_tml()

In [ ]:
a

In [ ]:
def get_final_relation_list(event_relations_with_ids):
  SLINKS=[elm.get('lid') for elm in myroot.findall('SLINK')]
  TLINKS=[elm.get('lid') for elm in myroot.findall('TLINK')]
  ALINKS=[elm.get('lid') for elm in myroot.findall('ALINK')]
  link_ids=set(SLINKS+ALINKS+TLINKS)
  link_ids=[int(link_id[1:]) for link_id in link_ids]
  for rel in event_relations_with_ids:
    rel[0]='l'+str(max(link_ids)+1)
    link_ids.append(max(link_ids)+1)
  return event_relations_with_ids